In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import plotly as pl

In [ ]:
sys.path.insert(0, "..")

np.random.random(20121020)

pl.offline.init_notebook_mode(connected=True)

In [ ]:
n_gene = 80

all_genes = pd.read_table("genes.txt", squeeze=True)

genes = np.random.choice(all_genes, size=n_gene, replace=False)

genes

In [ ]:
n_sample = 8

gene_x_sample = pd.DataFrame(
    np.random.random_sample(size=(n_gene, n_sample)),
    index=genes,
    columns=("Sample {}".format(i) for i in range(n_sample)),
)

gene_x_sample

In [ ]:
n_gene_set = 8

gene_sets = []

for i in range(n_gene_set):

    gene_sets.append(np.random.choice(genes, size=(i + 1) * 8, replace=False))

gene_sets = pd.DataFrame(
    gene_sets, index=tuple("Gene Set {}".format(i) for i in range(n_gene_set))
)

gene_sets

In [ ]:
from gsea.single_sample_gsea import single_sample_gsea

score = single_sample_gsea(gene_x_sample.iloc[:, 0] - 0.5, gene_sets.iloc[0])

score

In [ ]:
from gsea.nd_array.nd_array.normalize_nd_array import normalize_nd_array

for statistic in ("ks", "auc"):

    gene_score = gene_x_sample.iloc[:, 0]

    for normalization_method in (None, "0-1", "-0-"):

        if normalization_method is not None:

            gene_score = pd.Series(
                normalize_nd_array(gene_score, None, normalization_method),
                name=gene_score.name,
                index=gene_score.index,
            )

        single_sample_gsea(
            gene_score,
            gene_sets.iloc[0],
            statistic=statistic,
            title="statistic={} & normalization_method={}".format(
                statistic, normalization_method
            ),
        )

In [ ]:
from gsea.single_sample_gseas import single_sample_gseas

score__gene_set_x_sample = single_sample_gseas(gene_x_sample, gene_sets, n_job=2)

score__gene_set_x_sample

In [ ]:
from scipy.stats import pearsonr

from gsea.gsea import gsea

n_sample_per_phenotype = n_sample // 2

phenotypes = np.array((0,) * n_sample_per_phenotype + (1,) * n_sample_per_phenotype)


def function(x, y):

    return pearsonr(x, y)[0]


score_p_value = gsea(
    gene_x_sample,
    phenotypes,
    gene_sets.iloc[0, :],
    function,
    statistic="ks",
    n_permutation=10,
    permuting="gene",
    gene_score_name="Correlation",
)

score_p_value